In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

from ese.analysis.analyze_inf import load_cal_inference_stats
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/hdd/home/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

In [2]:
%%yaml results_cfg 

log:
    root: '/storage/vbutoi/scratch/ESE/inference'
    inference_groups: 
        # - '10_04_24_ISLES_3D_FineGrainTempSweep'
        - '10_07_24_ISLES_3D_TempSweep_wAugExamples'

options:
    verify_graceful_exit: True
    equal_rows_per_cfg_assert: False 

<IPython.core.display.Javascript object>

In [3]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=True
)

Finished loading inference stats.
Log amounts: log_root                                                                        log_set                                              
/storage/vbutoi/scratch/ESE/inference/10_07_24_ISLES_3D_TempSweep_wAugExamples  20241007_234139-JCNX-86f21dc4dc7bfd2107e87921cb01e48f    13000
                                                                                20241007_234144-W524-7ac8b7efd70f08ea7e659d9943b01dcb    13000
                                                                                20241007_234148-1JOH-ad20a399afbd325ab0ae5fdb62b3223b     6500
                                                                                20241007_234152-YKZT-979ecc4a85c631d5975637beb6bad3d9     6500
                                                                                20241007_234331-CFHF-42f3c43138003411004e5c7464869ce7    13000
                                                                                20241007_234335-F67B-d4c

In [4]:
# for ikey in inference_df:
#     print(ikey)

In [5]:
inference_df['experiment_model_dir'].unique()

array(['/storage/vbutoi/scratch/ESE/training/09_25_24_ISLES_3D_Dice_HeavyAug/20240925_234556-HGRQ-56cd3bf7df7a1b7d2a453b296d64407d'],
      dtype=object)

In [6]:
# Add necessary column and do a check to ensure that we only are dealing with one model.
def abs_vol_estimation_error(soft_volume, gt_volume):
    return np.abs(soft_volume - gt_volume)

inference_df.augment(abs_vol_estimation_error)
assert len(inference_df['experiment_model_dir'].unique()) == 1
exp_model_dir = inference_df['experiment_model_dir'].unique()[0]

# Gather the per-image optimal temperatures.

In [7]:
sub_cols_to_keep = [
    'abs_vol_estimation_error',
    'temperature',
    'data_id',
    'split'
]
# Filter out the columns we want to keep
individual_temp_df = inference_df[sub_cols_to_keep].drop_duplicates().reset_index(drop=True)
# Get the optimal temperature for each data_id
opt_temps_df = individual_temp_df.loc[individual_temp_df.groupby('data_id')['abs_vol_estimation_error'].idxmin()].reset_index(drop=True)

In [8]:
# We want to make a dictionary that maps from data_id to optimal temperature
data_id_to_opt_temp = dict(zip(opt_temps_df.data_id, opt_temps_df.temperature))

In [9]:
import json
# Now we are going to save this dictionary to a file in the exp_model_dir called 'opt_temps.'
opt_temp_file = os.path.join(exp_model_dir, 'opt_temps.json')
# Save the dictionary using json
with open(opt_temp_file, 'w') as f:
    json.dump(data_id_to_opt_temp, f)